<a href="https://colab.research.google.com/github/uba/tathu/blob/master/notebooks/full-example-goes16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Instalação TATHU - Tracking and Analysis of Thunderstorms**

Autor: Douglas Uba - douglas.uba@inpe.br

![https://github.com/uba/tathu](https://raw.githubusercontent.com/uba/tathu/master/docs/sphinx/img/logo-tathu.png)



Primeiro, instalamos o pacote TATHU e as dependências necessárias para a sua execução. 

Mais detalhes do processo de instalação podem ser vistos no notebook: [Instalação do TATHU e suas dependências.](https://github.com/uba/tathu/blob/master/notebooks/tathu-install.ipynb)

In [ ]:
!wget https://raw.githubusercontent.com/uba/tathu/master/notebooks/tathu-install.ipynb -O tathu-install.ipynb
%run tathu-install.ipynb

# **Obtenção dos Dados** 

### **Download de Imagens GOES-16**

Utilizando o suporte fornecido pelo TATHU, vamos realizar o download de um conjunto de imagens obtidas pelo satélite GOES-16. Mais especificamente, imagens do dia 26 de Outubro de 2022, canal 13 em 10,35 µm, horas [0-6] UTC, do setor *full-disk*. Essas imagens serão utilizadas para a detecção e rastreio dos objetos de interesse, os **Sistemas Convectivos (SC)**.

In [ ]:
from datetime import datetime
from tathu.downloader.goes import AWS
from tathu.progress import TqdmProgress

# Download 26 October 2022, Channel 13, [00, 01, 02, 03, 04, 05, 06] hours UTC
start = end = datetime.strptime('20221026', '%Y%m%d')
hours = ['00', '01', '02', '03', '04', '05', '06']

# From AWS (full-disk)
AWS.download(AWS.buckets['GOES-16'], ['ABI-L2-CMIPF'],
    start, end, hours, ['13'], './goes16-aws',
    progress=TqdmProgress('Download GOES-16 data (AWS)', 'files'))

### Verificando a Lista de Arquivos Obtidos

Os dados foram baixados. Vamos realizar uma busca no sistema de arquivos com o objetivo de definir uma lista de imagens que serão utilizados nos processos de detecção e rastreio. São arquivos do tipo netCDF, que possuem os valores medidos pelo sensor ABI a bordo do satélite GOES-16.



In [ ]:
import glob

# List download files
files = sorted(glob.glob("./goes16-aws/noaa-goes16/ABI-L2-CMIPF/2022/*/*/*.nc"))
for f in files:
  print(f)

### **Visualização Full-Disk**

Para exemplificar do tipo de dado que estamos trabalhando, vamos visualizar a imagem do dia 26 de Outubro de 2022, 00:00 UTC. Tratam-se de imagens no infravermelho termal, posicionado em uma região de janela atmosférica, em 10,35 µm. O setor *full-disk* refere-se ao imageamento de um Polo a outro da Terra, dentro dos limites de longitude estabelecidos pela posição do satélite geoestacionário.

In [ ]:
import matplotlib.pyplot as plt
from netCDF4 import Dataset

# Visualize first file
path = files[0]

# Open netCDF file and show full-disk data
nc = Dataset(path)
data = nc.variables['CMI'][:]
plt.imshow(data, cmap='Greys', vmin=180.0, vmax=320.0)
plt.colorbar()
nc.close()

### **Remapeamento para Grade Regular**

Para o processo de detecção e rastreio, precisamos remapear o dado  *full-disk* para uma grade regular. Essa operação pode ser realizada a partir do suporte fornecido pelo TATHU. Precisamos fornecer a região geográfica desejada e a resolução espacial que iremos trabalhar (em quilômetros). Aqui, fazemos o remapeamento do dado para a região geográfica que compreende a América do Sul, na resolução de 2 km.

In [ ]:
from osgeo import gdal

from tathu.constants import LAT_LONG_WGS84_SOUTH_AMERICA_EXTENT, LAT_LON_WGS84
from tathu.satellite import goes16

# Geographic area of regular grid [llx, lly, urx, ury], where ll=lower-left, ur=upper-right
extent = [-88.02, -46.50, -26.22, 12.54]

 # Grid resolution (kilometers)
resolution = 2.0

print('Remapping')
grid = goes16.sat2grid(path, extent, resolution, LAT_LON_WGS84, 'HDF5', progress=gdal.TermProgress_nocb)

# Visualize regular grid result
from tathu.visualizer import MapView

m = MapView(extent)
m.plotImage(grid, cmap='Greys', vmin=180.0, vmax=320.0)
m.show()


# **Detecção, Caracterização e Rastreio dos Sistemas Convectivos**

### Detecção Utilizando Limiarização

Vamos agora realizar a **detecção** dos SC utilizando um processo de limiarização, implementado na classe `detector.LessThan`.

Definimos alguns parâmetros de configuração, incluindo:

* `threshold` - Valor o máximo de temperatura de brilho (235K);
* `minarea` - Área mínima de detecção (3000km2) para cada SC.

In [ ]:
from tathu.tracking.utils import area2degrees

# Threshold value
threshold = 235 # Kelvin
    
# Define minimum area
minarea = 3000 # km^2

# Convert to degrees^2
minarea = area2degrees(minarea)

Realizamos o processo de deteção a partir da classe `detector.LessThan`:

In [ ]:
from tathu.tracking import detectors

# Create detector
detector = detectors.LessThan(threshold, minarea)

# Searching for systems
print('Searching for systems...')
systems = detector.detect(grid)
print('# Number of detected systems:', len(systems))

Os limites geográficos de cada SC detectado (em vermelho) podem ser visualizados a partir do seguinte trecho de código:


In [ ]:
m = MapView(extent)
m.plotImage(grid, cmap='Greys', vmin=180.0, vmax=320.0)
m.plotSystems(systems, facecolor='none', edgecolor='red', centroids=False)
m.show()

### **Extração de Atributos**

Uma vez definidos os limites geográficos de cada SC, podemos extrair uma série de atributos para cada objeto de interesse. 

Neste caso, pode-se considerar atributos espectrais (medidas de
um sensor em diferentes canais), análises estatísticas (média, variância,
etc) e de forma (tamanho, orientação, retangularidade), entre outros.

Neste exemplo, vamos extrair atributos estatísticos para cada SC, incluindo valor mínimo, média e desvio padrão de temperatura de brilho, além do número de pixels que compõe determinado sistema - `attrs = ['min', 'mean', 'std', 'count']`.

Essa operação é realizada pela classe `descriptors.StatisticalDescriptor`.



In [ ]:
# Attributes that will be computed
attrs = ['min', 'mean', 'std', 'count']

# Silence some warnings in order to improve visualization
import warnings
from shapely.errors import ShapelyDeprecationWarning
warnings.filterwarnings('ignore', category=ShapelyDeprecationWarning) 

from tathu.tracking import descriptors

# Create statistical descriptor
descriptor = descriptors.StatisticalDescriptor(rasterOut=True)

# Describe systems (stats)
descriptor.describe(grid, systems)

# Visualize attributes
for s in systems:
  print(s.name, s.attrs)

### **Função para Detecção e Extração de Atributos**

Compreendidos os conceitos de remapeamento, deteção e extração de atributos (i.e. caracterização), vamos agora definir uma função auxiliar em Python para realizar essas operações dado um caminho de arquivo - `path`.

Chamaremos a função de `detect`, definida a seguir. A função retorna ao final uma lista de objetos que representam cada SC.



In [ ]:
from tathu.utils import file2timestamp

def detect(path):
  # Extract time from file-name
  timestamp = file2timestamp(path, regex=goes16.DATE_REGEX, format=goes16.DATE_FORMAT).replace(microsecond=0)

  print('Processing', timestamp)

  # Remap data
  grid = goes16.sat2grid(path, extent, resolution, LAT_LON_WGS84, 'HDF5', progress=gdal.TermProgress_nocb)

  # Create detector
  detector = detectors.LessThan(threshold, minarea)

  # Searching for systems
  print('Searching for systems...')
  systems = detector.detect(grid)
  print('# Number of detected systems:', len(systems))

  # Adjust timestamp for each system
  for s in systems:
    s.timestamp = timestamp

  # Create statistical descriptor
  descriptor = descriptors.StatisticalDescriptor(rasterOut=True)

  # Describe systems (stats)
  descriptor.describe(grid, systems)

  # Free resources
  grid = None

  return systems

Agora podemos utilizar essa função de modo prático para cada arquivo de imagem.

Por exemplo, para a primeira imagem da lista temos:

In [ ]:
systems = detect(files[0])

## **Rastreio Utilizando Sobreposição de Áreas**

Definida a função `detect` capaz de detectar e extrair os atributos dos SC para cada instante de tempo, vamos agora tratar da **associação** desses elementos no tempo (i.e. **acompanhamento**, **rastreio automático**).

A etapa de rastreio é determinar como os objetos de interesse se comportaram no intervalo de tempo ∆t, decorrido
entre as duas imagens. Nessa etapa, os processos de detecção e caracterização também são utilizados. Em síntese, o acompanhamento deve ser capaz de determinar quais objetos da observação anterior ainda estão presentes no instante t (associação), além de também identificar novos objetos que surgiram.

Como exemplo, o rastreio pode ser realizado a partir da relação topológica entre os SC em conjunto com a análise das áreas de interseção.

![https://tathu.readthedocs.io/en/latest/split.merge.html](https://raw.githubusercontent.com/uba/tathu/master/docs/sphinx/img/system-events.png)

Situações consideradas durante o acompanhamento dos SC a partir do critério de interseção de área mínima: (a) continuidade, (b) split e (c) merge. As linhas tracejadas representam os sistemas no instante t−∆t.

Vamos utilizar o suporte fornecido pelo TATHU para realizar o rastreio considerando imagens de dois instantes de tempo distintos e consecutivos: 00:00 UTC e 00:10 UTC. Vamos utilizar a classe `tracker.OverlapAreaTracker`. Uma sobreposição de pelo menos 10% da área é necessária para associar os SC como sendo o mesmo elemento.



In [ ]:
from tathu.tracking import trackers

# Define overlap area criterion
overlapAreaCriterion = 0.1 # 10%

systems_00_00_UTC = detect(files[0]) # previous
systems_00_10_UTC = detect(files[1]) # current

# Create overlap area strategy
strategy = trackers.RelativeOverlapAreaStrategy(overlapAreaCriterion)

#  Let's track!
print('Tracking...')
t = trackers.OverlapAreaTracker(systems_00_00_UTC, strategy=strategy)
t.track(systems_00_10_UTC)
print('done!')

# Let's see the defined relations for each current system 
for system in systems_00_10_UTC:
  print(system.event)


De modo mais geral, podemos realizar o rastreio considerando todo o conjunto de imagens:


In [ ]:
from tathu.tracking import trackers

# Prepare tracking...
previous = None

# Define overlap area criterion
overlapAreaCriterion = 0.1 # 10%

# Create overlap area strategy
strategy = trackers.RelativeOverlapAreaStrategy(overlapAreaCriterion)

# for each image file
for f in files:
  # Detect current systems
  current = detect(f)

  if previous is None:
    previous = current
    continue

  # Let's track!
  print('Tracking...')
  t = trackers.OverlapAreaTracker(previous, strategy=strategy)
  t.track(current)
  print('done!')

  # Prepare next iteration
  previous = current